## Building the notebook in Google Colab with tensorflow

In [ ]:
!pip install opendatasets          
!pip install tensorflow_text              

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 28.2 MB/s 
     |████████████████████████████████| 588.3 MB 18 kB/s 
     |████████████████████████████████| 1.7 MB 83.3 MB/s 
     |████████████████████████████████| 439 kB 91.9 MB/s 
     |████████████████████████████████| 6.0 MB 85.1 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstal

In [ ]:
import opendatasets as od
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend as K
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, Flatten, LSTM
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
import tensorflow_hub as hub #1
import tensorflow_text as text #2 both are important for the hub
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import random
from datetime import datetime
import os

In [ ]:
od.download("https://www.kaggle.com/datasets/nkitgupta/jigsaw-regression-based-data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: omaratef3221
Your Kaggle Key: ··········


100%|██████████| 2.67G/2.67G [02:15<00:00, 21.1MB/s]


In [ ]:
data = pd.read_csv("/content/jigsaw-regression-based-data/train_data_version3.csv")

In [ ]:
data.head()

,text,y
0,oh why dear johnny boy part one again do you w...,2.08
1,savant13 is a fat shite,3.58
2,i am english i am i do not want to get blocked...,0.80
3,and why the fuck should i care g e t r i d o f...,4.58
4,yo dear sir james paul you a fruity ass nigga ...,2.08


In [ ]:
data["text"] = data["text"].astype(str)

In [ ]:
x = data.iloc[:,0]
y = data.iloc[:,1]
x_train, x_test, y_train, y_test = tts(x, y, test_size = 0.2) #Train test split

In [ ]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")  ## Preprocess padding and tokenizer
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")  ## Model
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text') #create input layer

preprocessed_text = bert_preprocess(text_input)

outputs = bert_encoder(preprocessed_text)
          
l = tf.keras.layers.Dropout(0.1, name="dropout", )(inputs = outputs['pooled_output'])
l = tf.keras.layers.Dense(1, name="output")(l)
model = tf.keras.Model(inputs=[text_input], outputs = [l])



Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss=rmse, metrics=['mae', 'mse', rmse])
model.fit(x_train, y_train, epochs=100, batch_size = 32)

Epoch 1/100
1200/1200 [==============================] - 96s 69ms/step - loss: 0.9541 - mae: 0.6633 - mse: 0.9867 - rmse: 0.9542
Epoch 2/100
1200/1200 [==============================] - 83s 69ms/step - loss: 0.8611 - mae: 0.5769 - mse: 0.8051 - rmse: 0.8612
Epoch 3/100
1200/1200 [==============================] - 83s 69ms/step - loss: 0.8287 - mae: 0.5547 - mse: 0.7446 - rmse: 0.8287
Epoch 4/100
1200/1200 [==============================] - 83s 69ms/step - loss: 0.8147 - mae: 0.5458 - mse: 0.7209 - rmse: 0.8146
Epoch 5/100
1200/1200 [==============================] - 82s 69ms/step - loss: 0.8008 - mae: 0.5374 - mse: 0.7001 - rmse: 0.8007
Epoch 6/100
1200/1200 [==============================] - 83s 69ms/step - loss: 0.7967 - mae: 0.5334 - mse: 0.6869 - rmse: 0.7966
Epoch 7/100
1200/1200 [==============================] - 83s 69ms/step - loss: 0.7930 - mae: 0.5304 - mse: 0.6835 - rmse: 0.7931
Epoch 8/100
1200/1200 [==============================] - 82s 68ms/step - loss: 0.7855 - mae: 0.52

In [ ]:
y_test = np.array(y_test)
predictions = model.predict(x_test)
print("RMSE: ", rmse(y_test, predictions).numpy())
print("MAE: ", mean_absolute_error(y_test, predictions))
print("MSE: ", mean_squared_error(y_test, predictions))

300/300 [==============================] - 21s 67ms/step
RMSE:  1.139733841987556
MAE:  0.4745842661057454
MSE:  0.5842067862471113


In [ ]:
def test(text):
  print("Text Toxicity level: ", model.predict(text, verbose = 0 )[0][0])
print(test(["he is nice"]))
print(test(["I want to kick his ass this stupid idiot"]))
print(test(["he is so fucking fat and dumbass"]))
print(test(["you are so lovely"]))

Text Toxicity level:  0.2873905
None
Text Toxicity level:  1.162678
None
Text Toxicity level:  1.9665953
None
Text Toxicity level:  0.76916987
None
